In [48]:
import pandas as pd
#from pandas_profiling import ProfileReport

In [49]:
train_data = pd.read_csv("../minicomp-rossman/data/train.csv").reset_index(drop=True)
store_data = pd.read_csv("../minicomp-rossman/data/store.csv").reset_index(drop=True)

/Users/sinarampe/opt/anaconda3/envs/mini_comp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
#profile = ProfileReport(store_data)
#profile

In [53]:
def add_time_features(df):
    df["Date"] = pd.to_datetime(df["Date"])
    df['Year'] = df.Date.dt.year - 2013
    df['Month'] = df.Date.dt.month
    df['Day'] = df.Date.dt.day
    df['DayOfWeek'] = df.Date.dt.dayofweek
    df['WeekOfYear'] = df.Date.dt.isocalendar().week.astype(int)
    df = df.drop(labels = "Date", axis=1)
    return df

In [54]:
def drop_nans_nulls_closed(df):
    df = df[df["Open"]==1]
    df = df[df["Sales"]!=0]
    df = df.dropna(subset = ["Store", "Open", "Promo", "StateHoliday", "SchoolHoliday", "Sales"])
    df["StateHoliday"].replace({0.0: "0"}, inplace=True)
    return df

In [55]:
def drop_cols(df):
    df = df.drop(labels = ["Customers"], axis=1)
    return df

In [56]:
train_data = add_time_features(train_data)
train_data = drop_nans_nulls_closed(train_data)
train_data = drop_cols(train_data)

In [57]:
merged_train = pd.merge(train_data, store_data, how='left', on="Store")

In [58]:
def recode(df, var_list=[]):
    map_dict = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    for v in var_list:
        df[v].replace(map_dict, inplace=True)
    return df

merged_train = recode(merged_train, ['StoreType', 'Assortment', 'StateHoliday'])

In [60]:
def month_to_str(df):
    map_dict = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    df['month_str'] = df.Month.map(map_dict)
    return df

merged_train = month_to_str(merged_train)

In [61]:
def check_promo(row):
    if isinstance(row['PromoInterval'],str) and row['month_str'] in row['PromoInterval']:
            return 1
    else:
            return 0

In [62]:
merged_train['PromoMonth'] =  merged_train.apply(lambda row: check_promo(row),axis=1)    

In [82]:
merged_train.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'month_str', 'PromoMonth', 'DayS'],
      dtype='object')

In [81]:
### TARGET ENC 
merged_train['Store'] = merged_train['Store'].map(merged_train.groupby('Store')['Sales'].mean())
merged_train['Month'] = merged_train['Month'].map(merged_train.groupby('Month')['Sales'].mean())
merged_train['DayS'] = merged_train['Day'].map(merged_train.groupby('Day')['Sales'].mean())

In [87]:
merged_train['DayOfWeekS'] = merged_train['DayOfWeek'].map(merged_train.groupby('DayOfWeek')['Sales'].mean())
merged_train['StoreType'] = merged_train['StoreType'].map(merged_train.groupby('StoreType')['Sales'].mean())
merged_train['Assortment'] = merged_train['Assortment'].map(merged_train.groupby('Assortment')['Sales'].mean())
merged_train['StoreM'] = merged_train['Store'].map(merged_train.groupby('Store')['Month'].mean())

In [88]:
merged_train.head()

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,month_str,PromoMonth,DayS,DayOfWeekS,StoreM
0,5160.810638,1,3139.0,1.0,0.0,1,1.0,0,6399.393606,1,...,NaN,1,14.0,2013.0,"Feb,May,Aug,Nov",Jan,0,7234.412655,6896.934074,6842.080243
1,12959.368313,1,2401.0,1.0,0.0,1,1.0,0,6399.393606,1,...,NaN,1,31.0,2013.0,"Jan,Apr,Jul,Oct",Jan,1,7234.412655,6896.934074,6854.443247
2,5173.130734,1,2646.0,1.0,0.0,1,1.0,0,6399.393606,1,...,NaN,1,5.0,2013.0,"Mar,Jun,Sept,Dec",Jan,0,7234.412655,6896.934074,6848.775611
3,7479.684100,1,3113.0,1.0,0.0,1,1.0,0,6399.393606,1,...,2011.0,0,NaN,NaN,NaN,Jan,0,7234.412655,6896.934074,6848.258186
4,4425.829741,1,2907.0,1.0,0.0,1,1.0,0,6399.393606,1,...,NaN,0,NaN,NaN,NaN,Jan,0,7234.412655,6896.934074,6847.086991


In [91]:
merged_train.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'month_str', 'PromoMonth', 'DayS',
       'DayOfWeekS', 'StoreM'],
      dtype='object')

In [89]:
train = merged_train.iloc[:int(0.9*train_data.shape[0]), :]
eva = merged_train.iloc[int(0.9*train_data.shape[0]):int(0.95*train_data.shape[0]), :]
test = merged_train.iloc[int(0.95*train_data.shape[0]):, :]

In [90]:
import xgboost as xgb
from xgboost import plot_importance

In [94]:
target = 'Sales'

my_features = ["Store",'Year', 'Month', 'Day', 'WeekOfYear', 'DayOfWeek','StateHoliday', 'SchoolHoliday', 
               'CompetitionDistance', 'Promo2', 'StoreType', 'Assortment', 'Open', 'Promo', "PromoMonth",
              'DayS', 'DayOfWeekS', 'StoreM']

In [95]:
merged_train[my_features].head()

,Store,Year,Month,Day,WeekOfYear,DayOfWeek,StateHoliday,SchoolHoliday,CompetitionDistance,Promo2,StoreType,Assortment,Open,Promo,PromoMonth,DayS,DayOfWeekS,StoreM
0,5160.810638,0,6399.393606,1,1,1,1,1.0,900.0,1,9902.088827,8141.902433,1.0,0.0,0,7234.412655,6896.934074,6842.080243
1,12959.368313,0,6399.393606,1,1,1,1,1.0,90.0,1,9902.088827,6567.665908,1.0,0.0,1,7234.412655,6896.934074,6854.443247
2,5173.130734,0,6399.393606,1,1,1,1,1.0,590.0,1,9902.088827,8141.902433,1.0,0.0,0,7234.412655,6896.934074,6848.775611
3,7479.684100,0,6399.393606,1,1,1,1,1.0,1260.0,0,9902.088827,6567.665908,1.0,0.0,0,7234.412655,6896.934074,6848.258186
4,4425.829741,0,6399.393606,1,1,1,1,1.0,18160.0,0,6841.678340,7118.831048,1.0,0.0,0,7234.412655,6896.934074,6847.086991


In [96]:
params = {"objective": "reg:squarederror", 
          "booster" : "gbtree", 
          "seed": 10 }

In [97]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def compute_rmspe(actual, prediction):
    """
    Computs RMSPE (root mean squared percentage error) between predictions from a model
    and the actual values of the target variable.
    """
    
    rmspe = sqrt(mean_squared_error(actual, prediction)) / prediction.mean()
    
    # rounding to 2 decimal places
    print('RMSPE is ', round(rmspe,2))
    
    return rmspe

In [98]:
def xgboost_experiment(vars_list, experiment_name, params, num_boost_round):
    dtrain = xgb.DMatrix(train[vars_list], label=train['Sales'], enable_categorical=True)
    deva = xgb.DMatrix(eva[vars_list], label=eva['Sales'])
    dtest = xgb.DMatrix(test[vars_list], label=test['Sales'], enable_categorical=True)
    
    #train
    xgb_model = xgb.train(params, dtrain, num_boost_round=num_boost_round, 
                      early_stopping_rounds=100, evals=[(deva, "Eval")], verbose_eval=False)

    # make prediction
    print('+++++ Results for experiment: ', experiment_name)
    pred = xgb_model.predict(dtest)
    print("Testerror")
    print(compute_rmspe(test[target], pred) / test[target].mean())
    pred = xgb_model.predict(dtrain)
    print("Trainerror")
    compute_rmspe(train[target], pred) 
    return xgb_model

In [99]:
#num_boost_round_list = [100, 500, 1000]

#for n in num_boost_round_list:
#    print('### Experiment with ', str(n), ' boosting rounds')
#   xgboost_experiment(my_features, "my_features", params, n)

In [100]:
my_model = xgboost_experiment(my_features, "final", params, 1000)

+++++ Results for experiment:  final
Testerror
RMSPE is  0.15
2.1595502176017674e-05
Trainerror
RMSPE is  0.09


## Make predictions on Holdout_data and create a submission.csv file

In [106]:
holdout_data = pd.read_csv("../minicomp-rossman/data/holdout_b29.csv")

In [107]:
holdout_data = holdout_data.rename(columns={"Unnamed: 0": "Id"})
holdout_data["Id"] = holdout_data["Id"] + 1

In [108]:
def drop_nn(df):
    df = df[df["Open"]==1]
    df = df.dropna(subset = ["Store", "Open", "Promo", "StateHoliday", "SchoolHoliday"])
    df["StateHoliday"].replace({0.0: "0"}, inplace=True)
    return df

In [109]:
holdout_data = add_time_features(holdout_data)
holdout_data = drop_nn(holdout_data)
holdout_data = drop_cols(holdout_data)

In [113]:
merged_holdout = pd.merge(holdout_data, store_data, how='left', on="Store")
merged_holdout = recode(merged_holdout, ['StoreType', 'Assortment', 'StateHoliday'])
merged_holdout = month_to_str(merged_holdout)
merged_holdout['PromoMonth'] =  merged_holdout.apply(lambda row: check_promo(row),axis=1)   

In [114]:
merged_holdout['Store'] = merged_holdout['Store'].map(merged_holdout.groupby('Store')['Sales'].mean())
merged_holdout['Month'] = merged_holdout['Month'].map(merged_holdout.groupby('Month')['Sales'].mean())
merged_holdout['DayS'] = merged_holdout['Day'].map(merged_holdout.groupby('Day')['Sales'].mean())
merged_holdout['DayOfWeekS'] = merged_holdout['DayOfWeek'].map(merged_holdout.groupby('DayOfWeek')['Sales'].mean())
merged_holdout['StoreType'] = merged_holdout['StoreType'].map(merged_holdout.groupby('StoreType')['Sales'].mean())
merged_holdout['Assortment'] = merged_holdout['Assortment'].map(merged_holdout.groupby('Assortment')['Sales'].mean())
merged_holdout['StoreM'] = merged_holdout['Store'].map(merged_holdout.groupby('Store')['Month'].mean())

KeyError: 'Column not found: Sales'